In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import csv
from io import StringIO
import json

In [2]:
url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=50&type=c%2c3%2c4%2c6%2c36%2c34%2c35%2c37%2c38%2c39%2c41%2c42%2c44%2c45%2c47%2c110%2c206%2c209&season=2019&month=0&season1=2019&ind=0&team=0&rost=0&age=0&filter=&players=0'

def get_page(url):
    response = get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

def get_fg_table(url):
    soup = get_page(url)
    max_page = soup.find('div', {"class": "rgWrap rgInfoPart"}).find_all('strong')[1].text
    pages = list(range(1,int(max_page)+1))
    table = soup.find('table',{"class": "rgMasterTable"})
    headers = []
    headers_website = table.find_all('tr')[1]
    for th in headers_website.find_all('th'):
        headers.append(th.text)
    output = StringIO()
    writer = csv.DictWriter(output, fieldnames=headers)
    writer.writeheader()
    for page in pages:
        page_url = url + '&page={}_50'.format(page)
        soup = get_page(page_url)
        table = soup.find('table',{"class": "rgMasterTable"})
        players = table.find_all('tr')[3:]
        for player in players:
            s = []
            for stat in player.find_all('td'):
                s.append(stat.text)
            s_dict = dict(zip(headers,s))
            writer.writerow(s_dict)
    output.seek(0)
    df = pd.read_csv(output)
    return df

fg_df = get_fg_table(url)
fg_df.head()

,#,Name,Team,Age,G,PA,BB/K,BB%,K%,OBP,SLG,OPS,BABIP,GB/FB,GB%,FB%,HR/FB,SwStr%,Pull%,Soft%
0,1,Cody Bellinger,Dodgers,23,60,259,1.00,13.9 %,13.9 %,0.456,0.721,1.177,0.367,0.70,27.0 %,38.4 %,28.2 %,9.5 %,51.6 %,13.4 %
1,2,Christian Yelich,Brewers,27,55,240,0.95,15.4 %,16.3 %,0.433,0.716,1.149,0.297,1.13,45.0 %,40.0 %,34.4 %,10.9 %,36.9 %,13.8 %
2,3,Mitch Garver,Twins,28,28,101,0.46,10.9 %,23.8 %,0.406,0.690,1.096,0.333,0.69,34.9 %,50.8 %,28.1 %,7.6 %,50.8 %,27.0 %
3,4,Austin Meadows,Rays,24,43,186,0.53,10.2 %,19.4 %,0.430,0.659,1.089,0.397,0.86,33.6 %,39.1 %,24.0 %,8.5 %,39.8 %,14.8 %
4,5,Anthony Rendon,Nationals,29,47,204,0.88,13.7 %,15.7 %,0.436,0.659,1.095,0.352,0.55,28.8 %,52.5 %,15.1 %,6.0 %,41.7 %,14.4 %


In [3]:
savant = 'https://baseballsavant.mlb.com/statcast_leaderboard?year=2019&abs=0&player_type=resp_batter_id'

def get_savant_table(url):
    soup = get_page(url)
    data = soup.find_all('script')[9].text
    data_string = data.split("var leaderboard_data = ",1)[1]
    cleaned = data_string.strip()[:-1]
    data_list = json.loads(cleaned)
    columns = list(data_list[0].keys())
    df = pd.DataFrame(data_list)
    df  = df[columns]
    return df

savant_df = get_savant_table(savant)
savant_df.head()

,name,attempts,max_hit_speed,min_hit_speed,avg_hit_speed,max_hit_angle,min_hit_angle,avg_hit_angle,anglesweetspot,anglesweetspotpercent,...,player_type,season,resp_batter_id,barrels,brl_percent,brl_pa,ev95plus,ev95percent,rowId,href
0,"Stanton, Giancarlo",4,120.6,93.0,107.7,19.7,-23.5,3.2,2,50.0,...,resp_batter_id,2019,519317,1,25.0,6.7,3,75.0,row_519317,"<a href=""/savant-player/519317"">Stanton, Gianc..."
1,"Guerrero Jr., Vladimir",100,118.9,54.4,91.3,69.0,-51.1,6.7,31,31.0,...,resp_batter_id,2019,665489,11,11.0,8.0,45,45.0,row_665489,"<a href=""/savant-player/665489"">Guerrero Jr., ..."
2,"Alonso, Pete",154,118.3,38.2,90.8,81.6,-87.2,13.6,60,39.0,...,resp_batter_id,2019,624413,31,20.1,12.5,68,45.6,row_624413,"<a href=""/savant-player/624413"">Alonso, Pete</a>"
3,"Sanchez, Gary",110,118.3,48.2,93.8,84.6,-67.0,22.9,37,33.6,...,resp_batter_id,2019,596142,28,25.5,15.8,56,50.9,row_596142,"<a href=""/savant-player/596142"">Sanchez, Gary</a>"
4,"Judge, Aaron",48,118.1,68.2,98.4,69.0,-38.6,13.0,20,41.7,...,resp_batter_id,2019,592450,9,18.8,10.1,32,66.7,row_592450,"<a href=""/savant-player/592450"">Judge, Aaron</a>"


In [4]:
def get_ids():
    csv = get('http://crunchtimebaseball.com/master.csv').content
    df = pd.read_csv(StringIO(csv.decode('latin-1')))
    return df

ids_df = get_ids()

In [5]:
new_fg = fg_df.merge(ids_df[['mlb_id', 'fg_name']], left_on = 'Name', right_on = 'fg_name', how = 'left')

In [6]:
merged = new_fg.merge(savant_df, left_on = 'mlb_id', right_on = 'resp_batter_id', how = 'left')

In [7]:
merged.head()

,#,Name,Team,Age,G,PA,BB/K,BB%,K%,OBP,...,player_type,season,resp_batter_id,barrels,brl_percent,brl_pa,ev95plus,ev95percent,rowId,href
0,1,Cody Bellinger,Dodgers,23,60,259,1.00,13.9 %,13.9 %,0.456,...,resp_batter_id,2019.0,641355.0,26,14.0,10.0,97,53.3,row_641355,"<a href=""/savant-player/641355"">Bellinger, Cod..."
1,2,Christian Yelich,Brewers,27,55,240,0.95,15.4 %,16.3 %,0.433,...,resp_batter_id,2019.0,592885.0,26,16.3,10.8,83,52.9,row_592885,"<a href=""/savant-player/592885"">Yelich, Christ..."
2,3,Mitch Garver,Twins,28,28,101,0.46,10.9 %,23.8 %,0.406,...,resp_batter_id,2019.0,641598.0,10,15.9,9.9,28,44.4,row_641598,"<a href=""/savant-player/641598"">Garver, Mitch</a>"
3,4,Austin Meadows,Rays,24,43,186,0.53,10.2 %,19.4 %,0.430,...,resp_batter_id,2019.0,640457.0,15,11.7,8.1,53,41.4,row_640457,"<a href=""/savant-player/640457"">Meadows, Austi..."
4,5,Anthony Rendon,Nationals,29,47,204,0.88,13.7 %,15.7 %,0.436,...,resp_batter_id,2019.0,543685.0,25,18.0,12.3,78,56.1,row_543685,"<a href=""/savant-player/543685"">Rendon, Anthon..."


In [8]:
merged['BB%'] = (merged['BB%'].str.replace('%','').astype(float)) / 100

In [9]:
merged.dtypes

#                          int64
Name                      object
Team                      object
Age                        int64
G                          int64
PA                         int64
BB/K                     float64
BB%                      float64
K%                        object
OBP                      float64
SLG                      float64
OPS                      float64
BABIP                    float64
GB/FB                    float64
GB%                       object
FB%                       object
HR/FB                     object
SwStr%                    object
Pull%                     object
Soft%                     object
mlb_id                     int64
fg_name                   object
name                      object
attempts                  object
max_hit_speed             object
min_hit_speed             object
avg_hit_speed             object
max_hit_angle             object
min_hit_angle             object
avg_hit_angle             object
anglesweet